<a href="https://colab.research.google.com/github/Kingfinalweapon/kings/blob/master/boat_racing_csv_k_%E5%8A%A0%E5%B7%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('f.csv', encoding="Shift-JIS")
df = df[['レースコード','レース名','距離(m)','天候','風向','風速(m)','波の高さ(cm)','決まり手','1着_着順','1着_登録番号','1着_展示タイム','1着_進入コース','1着_スタートタイミング','1着_レースタイム','2着_着順','2着_登録番号','2着_展示タイム','2着_進入コース','2着_スタートタイミング','2着_レースタイム','3着_着順','3着_登録番号','3着_展示タイム','3着_進入コース','3着_スタートタイミング','3着_レースタイム','4着_着順','4着_登録番号','4着_展示タイム','4着_進入コース','4着_スタートタイミング','4着_レースタイム','5着_着順','5着_登録番号','5着_展示タイム','5着_進入コース','5着_スタートタイミング','5着_レースタイム','6着_着順','6着_登録番号','6着_展示タイム','6着_進入コース','6着_スタートタイミング','6着_レースタイム']]
df.head()
del df['距離(m)']
del df['レース名']
df.head()

,レースコード,天候,風向,風速(m),波の高さ(cm),決まり手,1着_着順,1着_登録番号,1着_展示タイム,1着_進入コース,...,5着_展示タイム,5着_進入コース,5着_スタートタイミング,5着_レースタイム,6着_着順,6着_登録番号,6着_展示タイム,6着_進入コース,6着_スタートタイミング,6着_レースタイム
0,202008112401,晴,南,2,1,まくり差し,01,3843,7.00,3,...,7.05,5,0.11,. .,06,4747,7.04,1,0.29,. .
1,202008112402,晴,南,3,2,差し,01,4070,7.02,4,...,6.99,3,0.12,. .,06,3777,6.97,5,0.21,. .
2,202008112403,晴,南,2,1,差し,01,4424,6.97,4,...,6.97,1,0.14,. .,06,5009,7.05,6,0.10,. .
3,202008112404,晴,南,2,1,逃げ,01,4553,6.94,1,...,7.03,3,0.10,. .,06,3752,6.97,5,0.13,. .
4,202008112405,晴,南,2,1,差し,01,3944,6.99,2,...,7.07,3,0.14,. .,06,4193,6.92,6,0.14,. .


In [ ]:
# 横カラムのデータを縦カラムに加工
df_list = []
columns = ['rece_id','recers_id','determined','ex_time','ap_course','st','rece_time','weather','wind_d','wind_s','wave','rank']

df_result = pd.DataFrame()
for i in range(len(df)):
  for j in range(1,7):
      rece_id = df.iloc[i,0]
      recers_id = df.loc[i,'{}着_登録番号'.format(j)]
      determined = df.loc[i,'決まり手']
      ex_time = df.loc[i,'{}着_展示タイム'.format(j)]
      ap_course = df.loc[i,'{}着_進入コース'.format(j)]
      st = df.loc[i,'{}着_スタートタイミング'.format(j)]
      rece_time = df.loc[i,'{}着_レースタイム'.format(j)]
      weather = df.loc[i,'天候']
      wind_d = df.loc[i,'風向']
      wind_s = df.loc[i,'風速(m)']
      wave = df.loc[i,'波の高さ(cm)']
      rank = df.loc[i,'{}着_着順'.format(j)]

      df_list.extend([rece_id,recers_id, determined, ex_time,ap_course,st,rece_time,weather,wind_d,wind_s,wave,rank])
      df_before = pd.DataFrame(data =[df_list], columns =columns)
      df_list = []
      df_result = pd.concat([df_result,df_before],ignore_index=True)

df_result = df_result.dropna()
df_result = df_result.reset_index(drop=True)

In [ ]:
# determinedとweatherとwind_dをカテゴライズ
# https://koji30learn.com/python-boatrace-lightgbm/
import sklearn
from sklearn import preprocessing


#map関数でのエンコード
determined_mapping = {'逃げ':'1','差し':'2','まくり':'3','まくり差し':'4','抜き':'5','恵まれ':'6'}
df_result['determined'] = df_result['determined'].map(determined_mapping)

#ラベルエンコード 決まりて、天気、風向
list_LabelEncode = ['weather','wind_d']
for label in list_LabelEncode:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(df_result[label])
    df_result[label] = le.transform(df_result[label])
df_result.head()

,rece_id,recers_id,determined,ex_time,ap_course,st,rece_time,weather,wind_d,wind_s,wave,rank
0,202008112401,3843,4,7.0,3,0.11,1.50.6,0,3,2,1,01
1,202008112401,4164,4,7.0,2,0.18,1.52.0,0,3,2,1,02
2,202008112401,4248,4,7.06,4,0.11,1.53.9,0,3,2,1,03
3,202008112401,3906,4,6.98,6,0.16,1.54.0,0,3,2,1,04
4,202008112401,4213,4,7.05,5,0.11,. .,0,3,2,1,05


前処理

In [ ]:
# [rank]文字以外を削除
# df_result['rank'].astype('str').value_counts() カウント
df_result = df_result[~df_result['rank'].astype('str').str.contains('\D')]

# [rank]05→5など0を消す
rank_list = list(df_result['rank'])
rank_str = [str(i) for i in rank_list]
df_result['rank'] = [ele.lstrip('0') for ele in rank_str]

# 123=0,456=1
df_result['rank'] = df_result['rank'].astype(int).map(lambda x: 0 if x < 4 else 1)

# [rece_time]いったん欠地の扱いが微妙なので削除
del df_result['rece_time']

In [ ]:
df_result.sort_values('rece_id',ascending=True,inplace=True)
df_result.to_csv('boat_result.csv',index=False)